In [1]:
import codecs
from pathlib import Path
from typing import Callable, Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from scipy import sparse, stats
from sklearn.model_selection import train_test_split
import functools

In [4]:
def preprocess_dataset_new(dataset_name:str = 'amazon-book',group_num:int = 5,threshold: int = 1) -> Tuple:
    """Load and Preprocess datasets."""
    # load dataset.
    path = Path(f'../../data/{dataset_name}')
    train_file = str(path/'train.txt')
    test_file = str(path/'test.txt')

    num_items = 0
    num_users = 0
    n_train = 0
   
    with open(train_file) as f:
        for l in f.readlines():
            if len(l) >0:
                l = l.strip('\n').split(' ')
                items = [int(i) for i in l[1:]]
                uid = int(l[0])
                num_users = max(num_users,uid)
                if items != []:
                    num_items = max(num_items,max(items))
                n_train += len(items)
    
    with open(test_file) as f:
        for l in f.readlines():
            if len(l) >0:
                l = l.strip('\n')
                try:
                    items = [int(i) for i in l.split(' ')[1:]]
                except Exception:
                    continue
                uid = int(l[0])
                if len(items)>0:
                    num_items = max(num_items,max(items))
                num_users = max(num_users,uid)
                

    num_items = num_items+1
    num_users = num_users+1

    print(num_items)
    print(num_users)

    train = np.zeros((n_train,2))
    idx = 0 
    item_freq = np.zeros(num_items)
    user_freq = np.zeros(num_users)
    
    target_u = [[] for i in range(num_users)] 
    target_u_pop = [[[]for j in range(group_num)]for i in range(num_users)]
    candidate_u =[[] for i in range(num_users)]
    pos_u =[[] for i in range(num_users)]
    group_item = [[]for i in range(group_num)]
    all =set(list(range(num_items)))


    with open(train_file) as f:
        for l in f.readlines():
            if len(l) >0:
                l = l.strip('\n').split(' ')
                items = [int(i) for i in l[1:]]
                uid = int(l[0])
                num = len(items)
                user_freq[uid]+=num
                pos_u[uid] = items
                candidate_u[uid] = list(all - set(items))
                for i in range(num):
                    train[idx+i,0] = uid
                    train[idx+i,1] = items[i]
                    item_freq[items[i]] = item_freq[items[i]]+1       
                idx += num
    '''
    all_data = pd.DataFrame(
        np.zeros((num_users, num_items))).stack().reset_index()
    all_data = all_data.values[:, :2]
    #print(all_data.shape)
    
    unlabeled_data = np.array(
        list(set(map(tuple, all_data)) - set(map(tuple, train))), dtype=int)
    train = np.r_[np.c_[train, np.ones(train.shape[0])],
                  np.c_[unlabeled_data, np.zeros(unlabeled_data.shape[0])]]
    '''
    print('train shape')
    print(train.shape)
    
    # save datasets
    path = Path(f'../../data/{dataset_name}/point')
    path.mkdir(parents=True, exist_ok=True)
    
    path2 = Path(f'../../data/{dataset_name}/point/5')
    path2.mkdir(parents=True, exist_ok=True)
    np.save(str(path / 'train.npy'), arr=train.astype(np.int))
    np.save(str(path / 'item_freq.npy'), arr=item_freq)
    np.save(str(path / 'user_freq.npy'), arr=user_freq)
    
    #group_num=5
    ###############group devided#########################
    #group_mask = np.zeros((num_items,group_num))
    item_idx = item_freq.argsort()
    tot_pop = np.sum(item_freq)
    average_pop = tot_pop / group_num
    j = 0
    for i in range(group_num):
        cur_pop = 0
        while (cur_pop < average_pop) and (j<num_items):
            term_idx = item_idx[j]
            group_item[i].append(term_idx)
            j = j+1
            cur_pop = cur_pop +item_freq[term_idx]
    print('group done!')
   
############################################################
    
    with open(test_file) as f:
        for l in f.readlines():
            if len(l) >0:
                l = l.strip('\n')
                try:
                    items = [int(i) for i in l.split(' ')]
                except Exception:
                    continue
                uid = items[0]
                items = items[1:]
                #print(uid)
                target_u[uid] = items
                for k in range (group_num):
                    target_u_pop[uid][k]= list(set(items) & set(group_item[k])) 
            else:
                 break
    def cmp_by_item_freq(a,b):
            if(item_freq[a]>item_freq[b]):
                return -1
            elif(item_freq[a]<item_freq[b]):
                return 1
            else :
                return 0

    for u in range(num_users):
        target_u[u]= sorted(target_u[u],key = functools.cmp_to_key(cmp_by_item_freq))
    
    np.save(str(path/str(group_num)/'group_item.npy'),arr=np.array(group_item))
    np.save(str(path/str(group_num)/'target_u.npy'),arr=np.array(target_u))
    np.save(str(path/str(group_num)/'target_u_pop.npy'),arr=np.array(target_u_pop))
    np.save(str(path/str(group_num)/'candidate_u.npy'),arr=np.array(candidate_u))
    np.save(str(path/str(group_num)/'pos_u.npy'),arr=np.array(pos_u))


In [5]:
preprocess_dataset_new(dataset_name='ml-1m')

13096
160377
train shape
(2024510, 2)
group done!


<ipython-input-4-5ed66c9c3105>:140: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.save(str(path/str(group_num)/'group_item.npy'),arr=np.array(group_item))
<ipython-input-4-5ed66c9c3105>:141: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.save(str(path/str(group_num)/'target_u.npy'),arr=np.array(target_u))
<ipython-input-4-5ed66c9c3105>:142: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you